### ドライブのマウント

In [1]:
# Googleドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### ライブラリ・モジュールのインポート

In [2]:
# ライブラリの準備
!pip install timm
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as F
import torch.autograd as autograd
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from timm.scheduler import CosineLRScheduler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
import copy
import random
from PIL import Image
import pickle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.8 MB/s eta 0:00:00


### シード値の設定

In [3]:
# シード値を設定
def fix_seed(seed=1234):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

fix_seed(seed=1234)

### データセットの準備

In [4]:
# Colabのローカルファイルに自作データをコピー（初めからローカルファイルにアップロードする場合は不要）
!cp -r "/content/drive/MyDrive/HandSigns/" "/content/HandSigns/"

In [5]:
# 自作データをリサイズして、ファイル構造を保ったまま新しいフォルダに保存する関数を定義
def resize_images(base_dir, new_base_dir, image_size=(32, 32)):
    for folder in os.listdir(base_dir):
        folder_path = os.path.join(base_dir, folder)
        new_folder_path = os.path.join(new_base_dir, folder)
        # リサイズした自作データを保存する新しいフォルダを作成
        os.makedirs(new_folder_path, exist_ok=True)
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            # 拡張子の確認
            if file_path.endswith(('.jpg', '.jpeg')):
                # 画像を展開
                img = Image.open(file_path)
                # 画像をリサイズ
                img_resized = img.resize(image_size)
                # リサイズした画像を新しいフォルダに保存
                img_resized.save(os.path.join(new_folder_path, filename))

# 学習用の自作データのディレクトリを設定
train_base_dir = '/content/HandSigns/train'
# テスト用の自作データのディレクトリを設定
test_base_dir = '/content/HandSigns/test'

# リサイズした学習用の自作データを保存するディレクトリを設定
new_train_base_dir = '/content/HandSigns/resized_train_32'
# リサイズしたテスト用の自作データを保存するディレクトリを設定
new_test_base_dir = '/content/HandSigns/resized_test_32'
# リサイズする大きさを設定
image_size = (32, 32)

# 学習用の自作データをリサイズ
resize_images(train_base_dir, new_train_base_dir, image_size)
# テスト用の自作データをリサイズ
resize_images(test_base_dir, new_test_base_dir, image_size)

In [6]:
# 平均値と標準偏差を計算するためのHandSignsの前処理を定義
pre_transform_handsigns = transforms.Compose([
    transforms.ToTensor()
])

# 平均値と標準偏差を計算するためのHandSignsデータセットの前処理を定義
pre_train_dataset_handsigns = torchvision.datasets.ImageFolder("/content/HandSigns/resized_train_32", transform=pre_transform_handsigns)

# 平均値と標準偏差を計算するための変数を初期化
pre_mean_handsigns = 0.0
pre_std_handsigns = 0.0
pre_total_samples_handsigns = len(pre_train_dataset_handsigns)

# データセットのすべてのデータポイントに対して平均値と標準偏差を計算
for data in pre_train_dataset_handsigns:
    pre_image, _ = data
    pre_mean_handsigns += pre_image.mean(dim=(1, 2))  # テンソルのチャンネルごとに平均を計算
    pre_std_handsigns += pre_image.std(dim=(1, 2))    # テンソルのチャンネルごとに標準偏差を計算

# データセット全体の平均値と標準偏差を計算
pre_mean_handsigns /= pre_total_samples_handsigns
pre_std_handsigns /= pre_total_samples_handsigns

print("データセット全体の平均値: ", pre_mean_handsigns)
print("データセット全体の標準偏差: ", pre_std_handsigns)

データセット全体の平均値:  tensor([0.6954, 0.6754, 0.6450])
データセット全体の標準偏差:  tensor([0.1331, 0.1565, 0.1689])


In [7]:
# 学習用のHandSignsデータセットの前処理を定義
train_transform_handsigns = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=pre_mean_handsigns, std=pre_std_handsigns)
    ])

# テスト用のHandSignsデータセットの前処理を定義
test_transform_handsigns = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=pre_mean_handsigns, std=pre_std_handsigns)
    ])

# 学習用のHandSignsデータセットの読み込み
train_dataset_handsigns = torchvision.datasets.ImageFolder("/content/HandSigns/resized_train_32", transform=train_transform_handsigns)
# テスト用のHandSignsデータセットの読み込み
test_dataset_handsigns = torchvision.datasets.ImageFolder("/content/HandSigns/resized_test_32", transform=test_transform_handsigns)

# 学習用のHandSignsデータローダーを作成
train_loader_handsigns = torch.utils.data.DataLoader(dataset=train_dataset_handsigns, batch_size=16, shuffle=True, num_workers=2)
# テスト用のHandSignsデータローダーを作成
test_loader_handsigns = torch.utils.data.DataLoader(dataset=test_dataset_handsigns, batch_size=16, shuffle=False, num_workers=2)

In [8]:
# 学習データのクラス名とラベルの対応を確認
print(train_dataset_handsigns.class_to_idx)
# テストデータのクラス名とラベルの対応を確認
print(test_dataset_handsigns.class_to_idx)

{'100000': 0, '100001': 1, '100111': 2, '101000': 3, '110000': 4, '110001': 5, '111000': 6, '111001': 7, '111100': 8, '111111': 9}
{'100000': 0, '100001': 1, '100111': 2, '101000': 3, '110000': 4, '110001': 5, '111000': 6, '111001': 7, '111100': 8, '111111': 9}


In [9]:
# イテレータを作成
data_iter = iter(train_loader_handsigns)

# データを1バッチ分取得
images, labels = next(data_iter)

# 取得したバッチの画像とラベルの情報を確認
print("画像の形状:", images.shape)  # (バッチサイズ, チャンネル数, 高さ, 幅)
print("ラベル:", labels)

画像の形状: torch.Size([16, 3, 32, 32])
ラベル: tensor([3, 7, 4, 1, 3, 5, 6, 9, 9, 5, 3, 0, 4, 0, 2, 1])


## モデルの実装

In [10]:
# 1×1のサブネットワーク獲得用の畳み込みを定義
def supermaskconv1x1(in_channels, out_channels, stride=1):
    return SupermaskConv(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)

# 3×3のサブネットワーク獲得用の畳み込みを定義
def supermaskconv3x3(in_channels, out_channels, stride=1):
    return SupermaskConv(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)

In [11]:
# サブネットワーク獲得用のバッチ正則化として、非アフィン正規化を使用する（学習可能なパラメータを使用しない）
class NonAffineBatchNorm(nn.BatchNorm2d):
    def __init__(self, dim):
        super(NonAffineBatchNorm, self).__init__(dim, affine=False)

In [12]:
# 各重みにスコアを付与したスコアをソートしてtop k%を使用することでサブネットワークを獲得する
class GetSubnet(autograd.Function):
    @staticmethod
    def forward(ctx, scores, k):
        # スコアを複製する
        out = scores.clone()
        # スコアを昇順でソートする
        _, idx = scores.flatten().sort()
        # top k%以下のスコアの数
        j = int((1 - k) * scores.numel())
        # flat_outとoutは同じメモリを参照する（flat_outを変更するとoutにも影響する）
        flat_out = out.flatten()
        # top k%の要素を1にする
        flat_out[idx[j:]] = 1
        # top k%以外の要素を0にする
        flat_out[idx[:j]] = 0
        return out

    @staticmethod
    def backward(ctx, g):
        # 逆伝播時に勾配gをそのまま伝える
        return g, None

In [13]:
# サブネットワーク獲得用の畳み込みを定義
class SupermaskConv(nn.Conv2d):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # 重みと同じ形状のスコアを用意
        self.scores = nn.Parameter(torch.Tensor(self.weight.size()))
        # スコアを一様分布で初期化
        nn.init.kaiming_uniform_(self.scores, a=math.sqrt(5))
        # 重みの勾配を無効化する
        self.weight.requires_grad = False

    def _init_conv(self):
        # 重みを一様分布で初期化
        if self.init == 'kaiming_uniform':
            nn.init.kaiming_uniform_(self.weight, mode='fan_out', nonlinearity='relu')
        # 重みを正規分布で初期化
        elif self.init == 'kaiming_normal':
            nn.init.kaiming_normal_(self.weight, mode='fan_out', nonlinearity='relu')
        # 重みを符号つき定数で初期化
        elif self.init == 'signed_constant':
            fan = nn.init._calculate_correct_fan(self.weight, mode='fan_out')
            gain = nn.init.calculate_gain('relu')
            std = gain / math.sqrt(fan)
            self.weight.data = self.weight.data.sign() * std

    def set_init(self, init):
        # 初期化手法を設定
        self.init = init

    def set_prune_rate(self, prune_rate):
        # 刈り込み率を設定
        self.prune_rate = prune_rate

    @property
    def clamped_scores(self):
        # スコアとして非負の値を返すようにする（重要度を表す）
        return self.scores.abs()

    def forward(self, x):
        # サブネットワークを獲得
        subnet = GetSubnet.apply(self.clamped_scores, 1 - self.prune_rate)
        # サブネットワークでマスク
        w = self.weight * subnet
        x = F.conv2d(x, w, self.bias, self.stride, self.padding, self.dilation, self.groups)
        return x

In [14]:
# サブネットワーク獲得用のResidual Blocksを定義
class SupermaskBuildingBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.bn1 = NonAffineBatchNorm(in_channels)
        self.conv1 = supermaskconv3x3(in_channels, out_channels, stride)
        self.bn2 = NonAffineBatchNorm(out_channels)
        self.conv2 = supermaskconv3x3(out_channels, out_channels)
        self.dropout = nn.Dropout(p=0.3)
        self.relu = nn.ReLU(inplace=True)

        # 入力と出力のチャンネル数が異なる場合（strideが1より大きい場合）、ダウンサンプリング
        if in_channels != out_channels or stride > 1:
            self.shortcut = supermaskconv1x1(in_channels, out_channels, stride)
        else:
            self.shortcut = nn.Sequential()

    def forward(self, x):
        out = self.bn1(x)
        out = self.relu(out)
        out = self.conv1(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.conv2(out)
        # 残差写像と恒等写像の要素毎の和を計算
        out += self.shortcut(x)
        return out

In [15]:
class SupermaskWideResNet(nn.Module):
    def __init__(self, block, depth, k, num_classes=10):
        super().__init__()
        assert (depth - 4) % 6 == 0, "depth should be 6n + 4"
        n = (depth - 4) // 6
        channels = [16, 16 * k, 32 * k, 64 * k]
        self.conv1 = supermaskconv3x3(3, channels[0])
        # Residual Blocks（1)
        self.layer1 = self._make_layer(block, channels[0], channels[1], n)
        # Residual Blocks（2）
        self.layer2 = self._make_layer(block, channels[1], channels[2], n, stride=2)
        # Residual Blocks（3）
        self.layer3 = self._make_layer(block, channels[2], channels[3], n, stride=2)
        self.bn = NonAffineBatchNorm(channels[3])
        self.relu = nn.ReLU(inplace=True)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(channels[3], num_classes)

        # 重みの初期化
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                # 初期化手法を設定
                m.set_init(init)
                # 刈り込み率を設定
                m.set_prune_rate(prune_rate)
                # 重みを初期化
                m._init_conv()

    # Residual Blocksを作成する関数を定義
    def _make_layer(self, block, in_channels, out_channels, blocks, stride=1):
        layers = []
        # 最初の Residual Block（stride=stride）
        layers.append(block(in_channels, out_channels, stride))
        # 残りの Residual Block（stride=1）
        for _ in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

## 学習と評価（100 epochs, init=kaiming_uniform）

### prune_rate=0.999

In [16]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.999_uniform_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_HandSigns_from_CIFAR10_100epochs_pr0.999_uniform_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 10.0 %, Loss: 2.3044
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 10.0 %, Loss: 2.3092
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 8.8 %, Loss: 2.3023
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 8.4 %, Loss: 2.3003
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 8.5 %, Loss: 2.3062
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 8.5 %, Loss: 2.3099
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 7.7 %, Loss: 2.3182
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 8.6 %, Loss: 2.3156
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 7.7 %, Loss: 2.3038
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 7.7 %, Loss: 2.3079
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 8.5 %, Loss: 2.3112
Epoch [11/50], Learning Rate

In [17]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.999です


In [18]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [19]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [20]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 10.0 %


In [21]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.995

In [22]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.995_uniform_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_HandSigns_from_CIFAR10_100epochs_pr0.995_uniform_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 13.2 %, Loss: 2.2258
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 22.3 %, Loss: 2.2180
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 25.6 %, Loss: 1.8660
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 35.4 %, Loss: 1.6120
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 36.9 %, Loss: 2.3472
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 39.9 %, Loss: 1.6390
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 47.9 %, Loss: 1.9894
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 47.7 %, Loss: 1.5497
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 49.1 %, Loss: 1.0380
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 53.7 %, Loss: 2.0017
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 50.6 %, Loss: 2.1385
Epoch [11/50], Lear

In [23]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.995です


In [24]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [25]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [26]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 82.0 %


In [27]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.99

In [28]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.99_uniform_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_HandSigns_from_CIFAR10_100epochs_pr0.99_uniform_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 14.7 %, Loss: 2.3057
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 17.8 %, Loss: 2.2054
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 27.5 %, Loss: 1.6848
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 36.4 %, Loss: 1.8044
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 42.6 %, Loss: 1.5004
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 45.2 %, Loss: 0.6858
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 41.7 %, Loss: 1.3652
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 46.9 %, Loss: 2.9982
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 48.4 %, Loss: 2.2038
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 47.3 %, Loss: 1.7571
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 51.9 %, Loss: 2.4149
Epoch [11/50], Lear

In [29]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.990です


In [30]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [31]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [32]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 74.0 %


In [33]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.95

In [34]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.95_uniform_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_HandSigns_from_CIFAR10_100epochs_pr0.95_uniform_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 15.0 %, Loss: 2.1559
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 22.8 %, Loss: 2.1784
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 27.2 %, Loss: 2.2182
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 29.7 %, Loss: 2.0556
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 32.9 %, Loss: 2.0540
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 34.3 %, Loss: 1.3304
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 34.6 %, Loss: 1.9827
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 38.9 %, Loss: 1.8202
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 41.7 %, Loss: 0.7806
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 41.3 %, Loss: 1.6223
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 42.5 %, Loss: 1.6113
Epoch [11/50], Lear

In [35]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.950です


In [36]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [37]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [38]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 72.0 %


In [39]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.9

In [40]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.9_uniform_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_HandSigns_from_CIFAR10_100epochs_pr0.9_uniform_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 12.7 %, Loss: 2.1678
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 20.0 %, Loss: 2.2084
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 23.8 %, Loss: 2.0929
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 29.4 %, Loss: 2.3959
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 32.0 %, Loss: 1.2363
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 32.7 %, Loss: 1.6678
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 36.2 %, Loss: 2.4326
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 36.0 %, Loss: 1.7655
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 41.0 %, Loss: 2.3699
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 41.8 %, Loss: 2.2696
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 44.8 %, Loss: 3.1209
Epoch [11/50], Lear

In [41]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.900です


In [42]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [43]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [44]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 75.0 %


In [45]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.7

In [46]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.7_uniform_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_HandSigns_from_CIFAR10_100epochs_pr0.7_uniform_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 14.5 %, Loss: 2.2882
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 22.9 %, Loss: 2.1382
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 24.4 %, Loss: 2.0267
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 27.8 %, Loss: 1.6507
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 28.9 %, Loss: 2.0562
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 27.7 %, Loss: 2.1896
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 33.2 %, Loss: 2.6141
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 34.0 %, Loss: 1.9053
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 36.4 %, Loss: 1.9731
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 38.2 %, Loss: 2.5187
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 39.2 %, Loss: 1.3184
Epoch [11/50], Lear

In [47]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.700です


In [48]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [49]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [50]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 75.0 %


In [51]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.5

In [52]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.5_uniform_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_HandSigns_from_CIFAR10_100epochs_pr0.5_uniform_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 12.7 %, Loss: 2.2573
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 20.0 %, Loss: 2.1193
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 22.2 %, Loss: 1.7600
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 29.4 %, Loss: 1.6703
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 29.2 %, Loss: 3.0711
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 30.1 %, Loss: 1.3414
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 30.0 %, Loss: 1.8948
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 34.5 %, Loss: 1.9441
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 37.1 %, Loss: 1.7012
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 37.3 %, Loss: 2.2320
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 42.4 %, Loss: 2.5399
Epoch [11/50], Lear

In [53]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.500です


In [54]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [55]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [56]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 75.0 %


In [57]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

## 学習と評価（100 epochs, init=kaiming_normal）

### prune_rate=0.999

In [58]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.999_normal_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_HandSigns_from_CIFAR10_100epochs_pr0.999_normal_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 10.9 %, Loss: 2.2503
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 16.9 %, Loss: 2.2672
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 19.0 %, Loss: 2.1333
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 27.1 %, Loss: 2.0581
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 27.6 %, Loss: 1.4303
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 32.1 %, Loss: 1.7066
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 34.1 %, Loss: 2.6780
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 31.7 %, Loss: 2.6275
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 35.1 %, Loss: 2.0750
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 38.1 %, Loss: 1.6093
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 38.3 %, Loss: 1.6491
Epoch [11/50], Lear

In [59]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.999です


In [60]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [61]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [62]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 62.0 %


In [63]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.995

In [64]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.995_normal_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_HandSigns_from_CIFAR10_100epochs_pr0.995_normal_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 11.9 %, Loss: 2.2351
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 15.9 %, Loss: 2.2366
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 24.8 %, Loss: 2.1774
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 31.0 %, Loss: 1.9361
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 38.3 %, Loss: 1.4158
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 38.3 %, Loss: 1.1870
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 39.6 %, Loss: 2.7542
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 44.4 %, Loss: 1.6458
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 41.8 %, Loss: 1.2548
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 47.3 %, Loss: 2.4569
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 49.0 %, Loss: 1.8905
Epoch [11/50], Lear

In [65]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.995です


In [66]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [67]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [68]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 66.0 %


In [69]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.99

In [70]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.99_normal_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_HandSigns_from_CIFAR10_100epochs_pr0.99_normal_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 12.5 %, Loss: 2.2494
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 21.4 %, Loss: 2.2273
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 27.4 %, Loss: 1.7633
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 35.3 %, Loss: 2.2373
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 38.3 %, Loss: 1.9358
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 41.1 %, Loss: 1.9005
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 42.2 %, Loss: 1.7499
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 45.4 %, Loss: 0.8758
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 42.6 %, Loss: 1.5054
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 46.6 %, Loss: 1.0987
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 53.3 %, Loss: 0.9665
Epoch [11/50], Lear

In [71]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.990です


In [72]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [73]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [74]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 76.0 %


In [75]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.95

In [76]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.95_normal_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_HandSigns_from_CIFAR10_100epochs_pr0.95_normal_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 10.9 %, Loss: 2.1462
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 20.4 %, Loss: 2.1979
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 26.6 %, Loss: 1.8948
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 34.1 %, Loss: 1.4336
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 35.5 %, Loss: 1.7784
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 32.4 %, Loss: 2.4438
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 36.7 %, Loss: 2.6818
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 45.3 %, Loss: 1.4564
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 42.9 %, Loss: 1.9072
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 44.5 %, Loss: 1.9327
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 45.5 %, Loss: 1.3556
Epoch [11/50], Lear

In [77]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.950です


In [78]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [79]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [80]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 77.0 %


In [81]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.9

In [82]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.9_normal_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_HandSigns_from_CIFAR10_100epochs_pr0.9_normal_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 10.9 %, Loss: 2.3148
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 19.0 %, Loss: 2.1992
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 25.8 %, Loss: 2.0949
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 28.5 %, Loss: 2.2727
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 29.9 %, Loss: 2.2555
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 35.2 %, Loss: 1.5641
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 38.5 %, Loss: 1.3864
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 37.2 %, Loss: 1.5363
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 40.0 %, Loss: 1.5134
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 43.5 %, Loss: 2.0474
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 44.2 %, Loss: 1.2817
Epoch [11/50], Lear

In [83]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.900です


In [84]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [85]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [86]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 67.0 %


In [87]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.7

In [88]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.7_normal_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_HandSigns_from_CIFAR10_100epochs_pr0.7_normal_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 10.5 %, Loss: 2.2793
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 18.5 %, Loss: 2.1051
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 24.7 %, Loss: 1.8313
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 29.4 %, Loss: 2.4959
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 30.8 %, Loss: 1.5960
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 29.4 %, Loss: 2.1962
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 32.0 %, Loss: 3.2072
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 33.0 %, Loss: 2.2695
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 38.4 %, Loss: 1.3736
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 38.0 %, Loss: 1.8959
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 41.6 %, Loss: 2.3081
Epoch [11/50], Lear

In [89]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.700です


In [90]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [91]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [92]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 67.0 %


In [93]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.5

In [94]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.5_normal_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_HandSigns_from_CIFAR10_100epochs_pr0.5_normal_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 10.2 %, Loss: 2.2792
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 15.3 %, Loss: 2.2007
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 22.3 %, Loss: 2.0003
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 27.5 %, Loss: 2.5659
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 28.2 %, Loss: 1.5029
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 29.4 %, Loss: 2.3660
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 32.3 %, Loss: 3.0989
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 36.0 %, Loss: 1.6795
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 39.4 %, Loss: 2.1623
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 40.0 %, Loss: 1.7037
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 40.7 %, Loss: 1.7245
Epoch [11/50], Lear

In [95]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.500です


In [96]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [97]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [98]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 73.0 %


In [99]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

## 学習と評価（100 epochs, init=signed_constant）

### prune_rate=0.999

In [100]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.999_constant_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_HandSigns_from_CIFAR10_100epochs_pr0.999_constant_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 10.0 %, Loss: 2.3022
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 10.0 %, Loss: 2.3036
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 8.8 %, Loss: 2.2977
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 8.0 %, Loss: 2.3009
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 7.3 %, Loss: 2.3045
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 9.4 %, Loss: 2.3096
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 8.2 %, Loss: 2.3046
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 7.9 %, Loss: 2.3110
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 8.9 %, Loss: 2.3046
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 7.1 %, Loss: 2.3082
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 8.6 %, Loss: 2.3039
Epoch [11/50], Learning Rate

In [101]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.999です


In [102]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [103]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [104]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 10.0 %


In [105]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.995

In [106]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.995_constant_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_HandSigns_from_CIFAR10_100epochs_pr0.995_constant_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 15.9 %, Loss: 2.2778
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 18.8 %, Loss: 2.2367
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 27.2 %, Loss: 1.8281
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 33.6 %, Loss: 1.6149
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 40.4 %, Loss: 3.6689
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 43.3 %, Loss: 2.0949
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 46.2 %, Loss: 1.2277
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 48.6 %, Loss: 2.0840
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 49.5 %, Loss: 2.3063
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 52.9 %, Loss: 3.2350
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 49.4 %, Loss: 1.3347
Epoch [11/50], Lear

In [107]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.995です


In [108]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [109]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [110]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 86.0 %


In [111]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.99

In [112]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.99_constant_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_HandSigns_from_CIFAR10_100epochs_pr0.99_constant_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 11.1 %, Loss: 2.4033
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 18.8 %, Loss: 2.2118
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 25.5 %, Loss: 1.6970
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 32.2 %, Loss: 2.3823
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 38.7 %, Loss: 2.0276
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 40.7 %, Loss: 1.4890
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 40.6 %, Loss: 1.6131
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 45.7 %, Loss: 1.2501
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 45.4 %, Loss: 2.0851
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 47.4 %, Loss: 0.9883
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 49.5 %, Loss: 1.3555
Epoch [11/50], Lear

In [113]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.990です


In [114]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [115]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [116]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 78.0 %


In [117]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.95

In [118]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.95_constant_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_HandSigns_from_CIFAR10_100epochs_pr0.95_constant_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 15.4 %, Loss: 2.2170
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 21.7 %, Loss: 1.9506
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 27.9 %, Loss: 1.6224
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 32.1 %, Loss: 1.5595
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 32.2 %, Loss: 1.6114
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 35.2 %, Loss: 1.3436
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 37.4 %, Loss: 1.8826
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 37.9 %, Loss: 1.9725
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 41.1 %, Loss: 1.3834
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 40.8 %, Loss: 2.0802
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 45.1 %, Loss: 1.8893
Epoch [11/50], Lear

In [119]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.950です


In [120]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [121]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [122]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 73.0 %


In [123]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.9

In [124]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.9_constant_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_HandSigns_from_CIFAR10_100epochs_pr0.9_constant_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 14.2 %, Loss: 2.2650
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 20.3 %, Loss: 2.2476
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 27.1 %, Loss: 2.4817
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 27.0 %, Loss: 1.9663
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 31.5 %, Loss: 1.4870
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 31.8 %, Loss: 2.6257
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 36.5 %, Loss: 2.5305
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 37.0 %, Loss: 2.2079
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 39.8 %, Loss: 1.9980
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 40.8 %, Loss: 2.0157
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 45.3 %, Loss: 1.7506
Epoch [11/50], Lear

In [125]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.900です


In [126]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [127]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [128]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 64.0 %


In [129]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.7

In [130]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.7_constant_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_HandSigns_from_CIFAR10_100epochs_pr0.7_constant_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 13.0 %, Loss: 2.1645
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 22.1 %, Loss: 2.2418
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 28.3 %, Loss: 1.8301
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 29.4 %, Loss: 1.7447
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 31.1 %, Loss: 2.2097
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 31.1 %, Loss: 2.0309
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 33.5 %, Loss: 1.7491
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 35.3 %, Loss: 2.5909
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 41.1 %, Loss: 1.2838
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 41.7 %, Loss: 1.1772
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 42.8 %, Loss: 1.6256
Epoch [11/50], Lear

In [131]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.700です


In [132]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [133]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [134]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 74.0 %


In [135]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.5

In [136]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.5_constant_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_HandSigns_from_CIFAR10_100epochs_pr0.5_constant_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 11.1 %, Loss: 2.2687
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 19.5 %, Loss: 2.2196
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 23.4 %, Loss: 2.0489
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 29.0 %, Loss: 2.3512
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 29.7 %, Loss: 2.0972
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 32.4 %, Loss: 1.7371
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 34.1 %, Loss: 1.7872
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 37.7 %, Loss: 1.5647
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 39.0 %, Loss: 1.4475
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 34.4 %, Loss: 2.0731
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 44.3 %, Loss: 2.3665
Epoch [11/50], Lear

In [137]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.500です


In [138]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [139]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [140]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 71.0 %


In [141]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()